### Data Cleaning Script for Text Classification
This notebook cleans the text data by converting text to lowercase, removing extra whitespaces, and tokenizing the text.

In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
import string
from collections import Counter

# Download stopwords from nltk if not already available
nltk.download('stopwords')

# Define a set of stopwords (filler words)
stop_words = set(stopwords.words('english'))

# Function to clean text: Tokenize, lower, remove stopwords and punctuation
def clean_text(text):
    if not isinstance(text, str):
        return []
    # Tokenize and convert to lowercase
    words = text.lower().split()
    # Remove punctuation
    words = [word.translate(str.maketrans('', '', string.punctuation)) for word in words]
    # Remove stopwords
    words = [word for word in words if word not in stop_words]
    return words

# Function to find common words across all rows
def find_common_words_across_rows(cleaned_text_column):
    if cleaned_text_column.empty:
        return set()

    # Initialize with the set of words from the first row
    common_words = set(cleaned_text_column.iloc[0])

    # Perform intersection with the words in each subsequent row
    for row in cleaned_text_column.iloc[1:]:
        common_words.intersection_update(row)

    return common_words

# Main function to analyze the text data for a specific label
def analyze_text_from_excel(file_path, column_name, text_column='TEXT'):
    # Load the XLSX file into a DataFrame
    df = pd.read_excel(file_path)

    # Check if the column exists in the data
    if column_name not in df.columns:
        print(f"Column {column_name} does not exist in the data.")
        return

    # Print number of TRUE and FALSE values for the label
    label_counts = df[column_name].apply(lambda x: str(x).lower()).value_counts()
    true_count = label_counts.get('true', 0)
    false_count = label_counts.get('false', 0)

    print(f"\n\n----- Analyzing label: {column_name} -----")
    print(f"TRUE count: {true_count}, FALSE count: {false_count}\n")

    # Filter rows where the specified column contains only 'TRUE' or 'FALSE'
    df = df[df[column_name].apply(lambda x: str(x).lower() in ['true', 'false'])]

    # Filter rows where the column value is 'TRUE'
    true_df = df[df[column_name].apply(lambda x: str(x).lower() == 'true')]

    # Extract the text values from the filtered rows
    text_values = true_df[text_column]

    # Convert text values to DataFrame for processing
    text_df = pd.DataFrame({text_column: text_values})

    # Apply the clean_text function to each row
    text_df['cleaned_text'] = text_df[text_column].apply(clean_text)

    # Get common words across all rows
    common_words_across_rows = find_common_words_across_rows(text_df['cleaned_text'])

    # If there are no common words, find the most common words across all rows
    if not common_words_across_rows:
        # Concatenate all word lists (sum all the lists)
        all_words = [word for sublist in text_df['cleaned_text'] for word in sublist]

        if len(all_words) == 0:
            print(f"No valid words found for label {column_name}")
            return

        # Count occurrences of each word
        word_counts = Counter(all_words)

        # Get the top N most common words
        most_common_words = word_counts.most_common()

        # Display the most common words
        print(f"Most common words for label {column_name}:")
        print(most_common_words)
    else:
        # Display the common words across all rows
        print(f"Common words for label {column_name}: {common_words_across_rows}")

# Run the analysis for multiple labels

file_path = '/content/prod_silver.xlsx'

# Labels to analyze
columns_to_analyze = [
    'Obscenity and Profanity, including language, gestures, and explicitly gory, graphic or repulsive content intended to shock and disgust',
    'Safe',
    'Death, Injury or Military Conflict',
    'Arms and Ammunition',
    'Debated Sensitive Social Issues',
    'Hate Speech & Acts of aggression',
    'Adult& Explicit Sexual Content',
    'Crime & Harmful acts to individuals and Society and Human Right Violation',
    'Illegal Drugs/Tobacco/e-cigarettes/Vaping/Alcohol'
]

# Loop over each label and analyze the text
for column_name in columns_to_analyze:
    analyze_text_from_excel(file_path, column_name)


In [ ]:
import pandas as pd
import os

def modify_excel(input_file, output_file):
    # Read the Excel file
    df = pd.read_excel(input_file)

    # Make all columns empty except column #4 (index 3 in 0-based indexing)
    for col in df.columns:
        if df.columns.get_loc(col) != 3:  # Keep column #4 (index 3) unchanged
            df[col] = ''

    # Remove columns 1, 2, 3, and 5 (0-based index)
    columns_to_drop = [0, 1, 2, 4]  # Python uses 0-based indexing
    df = df.drop(df.columns[columns_to_drop], axis=1)

    # Write the modified data to a single sheet in a new Excel file
    df.to_excel(output_file, index=False)

# Set the input file path
file_path = '/content/prod_silver.xlsx'

# Check if the file exists
if not os.path.exists(file_path):
    print(f"Error: The file '{file_path}' does not exist.")
else:
    # Generate output file name
    output_file = os.path.join(os.path.dirname(file_path), "modified_" + os.path.basename(file_path))

    # Run the function
    modify_excel(file_path, output_file)
    print(f"Process completed. Output file created: {output_file}")
